# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [60]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders=pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
orders.columns
orders=orders.drop(columns='Unnamed: 0')

In [4]:
# your code here
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [5]:
orders.shape

(397924, 13)

In [208]:
#orders['CustomerID'].unique()
#orders['CustomerID'].nunique()
#orders['CustomerID'].value_counts(dropna=False)
#orders['CustomerID'].isna().sum()

note the lenght of customerid value counts = 4339. compare to aggregation to confirm it is done correctly. 

Sub Problem 1: How to aggregate the amount_spent for unique customers?

In [20]:
orders.groupby(['CustomerID']).agg({'amount_spent':sum})

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?

In [18]:
summary=orders.describe().T
display(summary)

,count,mean,std,min,25%,50%,75%,max
InvoiceNo,397924.0,560617.126645,13106.167695,536365.0,549234.00,561893.00,572090.00,581587.00
year,397924.0,2010.934259,0.247829,2010.0,2011.00,2011.00,2011.00,2011.00
month,397924.0,7.612537,3.416527,1.0,5.00,8.00,11.00,12.00
day,397924.0,3.614555,1.928274,1.0,2.00,3.00,5.00,7.00
hour,397924.0,12.728247,2.273535,6.0,11.00,13.00,14.00,20.00
Quantity,397924.0,13.021823,180.420210,1.0,2.00,6.00,12.00,80995.00
UnitPrice,397924.0,3.116174,22.096788,0.0,1.25,1.95,3.75,8142.75
CustomerID,397924.0,15294.315171,1713.169877,12346.0,13969.00,15159.00,16795.00,18287.00
amount_spent,397924.0,22.394749,309.055588,0.0,4.68,11.80,19.80,168469.60


In [21]:
orders_grouped = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent':sum})

In [22]:
q25 = np.quantile(orders_grouped.amount_spent, q=.25)
q50 = np.quantile(orders_grouped.amount_spent, q=.50)
q75 = np.quantile(orders_grouped.amount_spent, q=.75)
q95 = np.quantile(orders_grouped.amount_spent, q=.95)

def select_quantiles(value):
    if value < q25:
        return "q25"
    elif value < q50:
        return "q50"
    elif value < q75:
        return "q75"
    elif value > q75 and value<q95:
        return "Preferred Customers"
    else:
        return "VIP" 

In [23]:
orders_grouped['quantiles'] = orders_grouped.amount_spent.apply(select_quantiles)

In [24]:
orders_grouped.reset_index(inplace=True)
orders_grouped

,CustomerID,Country,amount_spent,quantiles
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred Customers
2,12348,Finland,1797.24,Preferred Customers
3,12349,Italy,1757.55,Preferred Customers
4,12350,Norway,334.40,q50
...,...,...,...,...
4342,18280,United Kingdom,180.60,q25
4343,18281,United Kingdom,80.82,q25
4344,18282,United Kingdom,178.05,q25
4345,18283,United Kingdom,2094.88,Preferred Customers


# Question 1 combined: IDENTIFYING VIP AND PREFERRED CUSTOMERS:

In [38]:
VIPandPC=orders_grouped[(orders_grouped['quantiles']=='VIP')|(orders_grouped['quantiles']=='Preferred Customers')] 
display(VIPandPC)

,CustomerID,Country,amount_spent,quantiles
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred Customers
2,12348,Finland,1797.24,Preferred Customers
3,12349,Italy,1757.55,Preferred Customers
5,12352,Norway,2506.04,Preferred Customers
...,...,...,...,...
4327,18259,United Kingdom,2338.60,Preferred Customers
4328,18260,United Kingdom,2643.20,Preferred Customers
4336,18272,United Kingdom,3078.58,Preferred Customers
4345,18283,United Kingdom,2094.88,Preferred Customers


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [197]:
vipcountry=orders_grouped[(orders_grouped['quantiles']=='VIP')]
display(vipcountry)

,CustomerID,Country,amount_spent,quantiles
0,12346,United Kingdom,77183.60,VIP
10,12357,Switzerland,6207.67,VIP
12,12359,Cyprus,6372.58,VIP
52,12409,Switzerland,11072.67,VIP
57,12415,Australia,124914.53,VIP
...,...,...,...,...
4215,18109,United Kingdom,8052.97,VIP
4237,18139,United Kingdom,8438.34,VIP
4261,18172,United Kingdom,7561.68,VIP
4300,18223,United Kingdom,6484.54,VIP


In [198]:
vipcountry=vipcountry.rename(columns={'quantiles':'VIP_Status'})

display(vipcountry)

,CustomerID,Country,amount_spent,VIP_Status
0,12346,United Kingdom,77183.60,VIP
10,12357,Switzerland,6207.67,VIP
12,12359,Cyprus,6372.58,VIP
52,12409,Switzerland,11072.67,VIP
57,12415,Australia,124914.53,VIP
...,...,...,...,...
4215,18109,United Kingdom,8052.97,VIP
4237,18139,United Kingdom,8438.34,VIP
4261,18172,United Kingdom,7561.68,VIP
4300,18223,United Kingdom,6484.54,VIP


In [199]:
def countrycount(x):
    if x =="VIP":
        return 1

In [200]:
vipcountry['Most_VIPs']=vipcountry.VIP_Status.apply(countrycount)
#display(vipcountry)

In [202]:
vipcountry=vipcountry.groupby(['Country']).agg({'Most_VIPs':sum})
#display(vipcountry)

In [204]:
specialstatusVIPCountry=vipcountry[vipcountry['Most_VIPs']==vipcountry['Most_VIPs'].max()]
print(specialstatusVIPCountry)

                Most_VIPs
Country                  
United Kingdom        178


In [176]:
#specialstatuscountry=vipcountry[vipcountry['Most_VIPs']==vipcountry['Most_VIPs'].max()]
#print(specialstatuscountry)

Initially i misunderstood question 2 and I answered a different question as shown below. i am going to keep it though because it is cool! and would like to keep it for reference. SKIP TO QUESTION 3 PLEASE. 

In [177]:
mostprofitablecountry=VIPandPC.groupby(['Country']).agg({'amount_spent':max})
#display(mostprofitablecountry)

In [178]:
maxspent = mostprofitablecountry['amount_spent'].max()

def maxspentcountry(value):
    if value ==maxspent:
        return 'TOP'
    

In [179]:
mostprofitablecountry['Mostprofitable'] = mostprofitablecountry.amount_spent.apply(maxspentcountry)
display (mostprofitablecountry)

,amount_spent,Mostprofitable
Country,,
Australia,124914.53,None
Austria,2662.06,None
Belgium,5226.23,None
Canada,2940.04,None
Channel Islands,8137.02,None
Cyprus,6372.58,None
Denmark,7829.89,None
EIRE,143825.06,None
Finland,7956.46,None


In [180]:
mostprofitablecountry[mostprofitablecountry['Mostprofitable']=='TOP']

,amount_spent,Mostprofitable
Country,,
Netherlands,280206.02,TOP


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [181]:
VIPandPC=orders_grouped[(orders_grouped['quantiles']=='VIP')|(orders_grouped['quantiles']=='Preferred Customers')] 
display(VIPandPC)

,CustomerID,Country,amount_spent,quantiles
0,12346,United Kingdom,77183.60,VIP
1,12347,Iceland,4310.00,Preferred Customers
2,12348,Finland,1797.24,Preferred Customers
3,12349,Italy,1757.55,Preferred Customers
5,12352,Norway,2506.04,Preferred Customers
...,...,...,...,...
4327,18259,United Kingdom,2338.60,Preferred Customers
4328,18260,United Kingdom,2643.20,Preferred Customers
4336,18272,United Kingdom,3078.58,Preferred Customers
4345,18283,United Kingdom,2094.88,Preferred Customers


In [191]:
VIPandPCountries=VIPandPC.copy()
#display(VIPandPCountries)


In [192]:
VIPandPCountries=VIPandPCountries.rename(columns={'quantiles':'VIP_PC'})
#display(VIPandPCountries)

In [189]:
def countrycountVIPPC(x):
    if x =="VIP" or x=='Preferred Customers':
        return 1

In [193]:
VIPandPCountries['Most_VIPsPlustPC']=VIPandPCountries.VIP_PC.apply(countrycountVIPPC)
display(VIPandPCountries)

,CustomerID,Country,amount_spent,VIP_PC,Most_VIPsPlustPC
0,12346,United Kingdom,77183.60,VIP,1
1,12347,Iceland,4310.00,Preferred Customers,1
2,12348,Finland,1797.24,Preferred Customers,1
3,12349,Italy,1757.55,Preferred Customers,1
5,12352,Norway,2506.04,Preferred Customers,1
...,...,...,...,...,...
4327,18259,United Kingdom,2338.60,Preferred Customers,1
4328,18260,United Kingdom,2643.20,Preferred Customers,1
4336,18272,United Kingdom,3078.58,Preferred Customers,1
4345,18283,United Kingdom,2094.88,Preferred Customers,1


In [194]:
VIPandPCountries=VIPandPCountries.groupby(['Country']).agg({'Most_VIPsPlustPC':sum})
display(VIPandPCountries)

,Most_VIPsPlustPC
Country,
Australia,4
Austria,1
Belgium,11
Canada,1
Channel Islands,4
Cyprus,3
Denmark,3
EIRE,3
Finland,5


In [207]:
specialstatusVIPPCCountry=VIPandPCountries[VIPandPCountries['Most_VIPsPlustPC']==VIPandPCountries['Most_VIPsPlustPC'].max()]
print(specialstatusVIPPCCountry)

                Most_VIPsPlustPC
Country                         
United Kingdom               934
